In [57]:
from tweepy import OAuthHandler, API, Stream, Cursor
from tweepy.streaming import StreamListener
from urllib3.exceptions import ProtocolError
import json, time
from TweetData import keys
import pandas as pd

In [ ]:
# Autenticación con el API de Twitter
auth = OAuthHandler(keys.consumer_key, keys.consumer_secret)
auth.set_access_token(keys.access_token, keys.access_token_secret)
# Establecer la conexión con el API con la autenticación
api = API(auth, wait_on_rate_limit=True)

In [34]:
# Definir la búsqueda a realizar
search_word = "trafico panama"

start_time = time.time()
# Obtener los tweets de los últimos 7 días
cursor = Cursor(api.search,
                q=search_word,
                # since='2021-04-23',
                tweet_mode='extended').items()

# Recorrer el cursor
tweets = [t for t in cursor]
end_time = time.time()

print('Duración: %.2f s' % (end_time-start_time))
print('Tweets obtenidos: %i' % len(tweets))

Duración: 18.35 s
Tweets obtenidos: 313


In [69]:
# Objeto Status con su atributo _json
tweets[0]._json

In [70]:
# Declarar los atributos que se desean obtener
attributes = ['id', 'full_text', 'created_at', 'retweet_count', 'favorite_count']
# Traer los valores por comprensión de listas
data = [[tweet._json[key] for key in attributes] for tweet in tweets]

In [71]:
df = pd.DataFrame(data, columns=attributes)

In [77]:
df[['retweet_count', 'favorite_count']].sum()

retweet_count     2317
favorite_count     444
dtype: int64

In [ ]:
class SListener(StreamListener):

    # initialize API
    def __init__(self, api=None, fprefix='streamer'):
        self.api = api or API()
        # set db connection
        # self.db = DB_Connection()

    # for each tweet streamed
    def on_status(self, status):
        print(status)

    # if theres an error
    def on_error(self, status_code):
        if status_code == 420:
            # Returning False in on_data disconnects the stream
            return False

In [ ]:
listen = SListener(api)
stream = Stream(auth, listen)

In [ ]:
while True:
    # maintian connection unless interrupted
    try:
        keywords = ['@traficocpanama,traficocpanama,trafico panama']
        stream.filter(track=keywords)
    # reconnect automantically if error arise
    # due to unstable network connection
    except (ProtocolError, AttributeError):
        continue